In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
import sys
from sklearn import linear_model
from sklearn import metrics

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [4]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [5]:
#READING

In [6]:
#baseline approach: creates the set of popular books, with a threshold determined from hw3
def popularSet():
    # Copied from baseline code
    bookCount = defaultdict(int)  #all books
    totalRead = 0                 #how many books were read

    for user,book,_ in readCSV("train_Interactions.csv.gz"):
        bookCount[book] += 1
        totalRead += 1

    #list of (reads per book, book id), sorted by most popular
    mostPopular = [(bookCount[x], x) for x in bookCount] 
    mostPopular.sort()
    mostPopular.reverse()

    #return the set of books such that half of all ratings were for these books
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalRead / 1.39: break #threshold of 1.39 determined from hw3 
            
    return return1

In [7]:
def Jaccard(s1, s2):
    intersection_size = len(s1.intersection(s2))
    union_size = len(s1.union(s2))
    if union_size == 0:
        return 0
    return intersection_size / union_size

In [8]:
#create the ratingsPerUser and ratingsPerItem
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in readCSV("train_Interactions.csv.gz"):
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

    
#create the popular set:
popular = popularSet()

In [9]:
### Would-read: based on hw3, q4: use the baseline's approach of checking if the book is popular, or if the similar users have read the book 
#predictions = open("predictions_Read.csv", 'w')

#for l in open("pairs_Read.csv"):
#    if l.startswith("userID"):
        #header
#        predictions.write(l)
#        continue
        
    #get the user and the book
#    user,book = l.strip().split(',')
    
    #get all books b' that a user has read
#    other_books = [b for (b, _) in ratingsPerUser[user]]

    #set of ratings for target_book
#    targetSet = set([u for (u,_) in ratingsPerItem[book]])
#    maxSim = 0.0
    
    #get the max similarity for the target_book and user's books
#    for other in other_books:
#        otherSet = set([u for (u,_) in ratingsPerItem[other]])

#        sim = Jaccard(targetSet, otherSet)
#        maxSim = sim if (sim > maxSim) else maxSim
        
        
#    if b in popular:
#        predictions.write(user + ',' + book + ",1\n")
#    elif maxSim > 0.005:   #threshold determined from hw3
#        predictions.write(user + ',' + book + ",1\n")
#    else:
#        predictions.write(user + ',' + book + ",0\n")

#predictions.close()


In [ ]:
def predictRatingFromUserJaccard(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerItem[item]:
        u2 = d['user_id']
        if u2 == user: continue
        ratings.append(d['rating'] - userAverages[u2])
        similarities.append(Jaccard(itemsPerUser[user],itemsPerUser[u2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return userAverages[user] + sum(weightedRatings) / sum(similarities)
    else:
        return ratingMean
    
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue

In [10]:
##CATEGORIES

In [11]:
#prepare category data and count occurrences of words
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)
    
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1
        
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [12]:
#get the longer words
longerWords = [w[1] for w in counts[:1500] if len(w[1]) > 3]
wordId = dict(zip(longerWords, range(len(longerWords))))
wordSet = set(longerWords)

In [13]:
#method to make feature matrix, using review_text
def feature(datum):
    feat = [0]*len(wordSet)
    t = datum['review_text']
    t = t.lower() # lowercase string
    t = [c for c in t if not (c in string.punctuation)] # non-punct characters
    t = ''.join(t) # convert back to string
    words = t.strip().split() # tokenizes
    for w in words:
        if not (w in wordSet): continue
        feat[wordId[w]] += 1
    feat.append(1)
    return feat

In [14]:
#train the model
X = [feature(d) for d in data]
y = [d['genreID'] for d in data]

In [1]:
mod = linear_model.LogisticRegression()
mod.fit(X, y)

NameError: name 'linear_model' is not defined

In [ ]:
len(longerWords)

In [ ]:
#get the test data for category
testCatData = []
for l in readGz("test_Category.json.gz"):
    testCatData.append(l)

#create the test matrix and get predictions
testX = [feature(d) for d in testCatData]
predicted = mod.predict(testX)

In [ ]:
len(predicted)

In [ ]:
predictions = open("predictions_Category.csv", 'w')
predictions.write("userID,reviewID,prediction\n")
for i in range(len(testCatData)):
    predictions.write(testCatData[i]['user_id'] + ','
                      + testCatData[i]['review_id'] + ","
                      + str(predicted[i]) + "\n")
